# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [20]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 200)

import sys
import os
import re
from sqlalchemy import create_engine
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from scipy.stats import gmean
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse_table', engine)
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [5]:
# Descriptive statistics to verify dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
id                        26216 non-null int64
message                   26216 non-null object
original                  10170 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null i

In [6]:
df.shape

(26216, 40)

In [7]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
#Removing child_alone  column as it is not meaningful, the column has just 0 on its values
df = df.drop(['child_alone'],axis=1)

The column 'Related' has a max value of '2', which could be an error as this column looks like a column with binominal values

In [9]:
# Checking count of '2' on related column
df.groupby('related').count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,6643,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906
2,188,188,132,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188


In [10]:
# Replacing value '2' with '1' on related column
df['related'] = df['related'].map(lambda x: 1 if x==2 else x)
df.groupby('related').count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,20094,20094,6775,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094


In [11]:
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [17]:
# Creating custom transformer to be used in the ML pipeline

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
def ML_pipeline(clf = AdaBoostClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline

# include StartingVerbExtractor custom transformer
def ML_pipeline_2(clf = AdaBoostClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [21]:
# Splitting data and training pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y)

model = ML_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
# Testing model and verifying results

y_pred_test = model.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5012
               request       0.75      0.56      0.64      1093
                 offer       0.17      0.04      0.06        28
           aid_related       0.77      0.60      0.67      2717
          medical_help       0.63      0.27      0.38       546
      medical_products       0.58      0.35      0.43       307
     search_and_rescue       0.57      0.23      0.33       179
              security       0.16      0.03      0.05       106
              military       0.60      0.33      0.42       232
                 water       0.72      0.62      0.67       411
                  food       0.79      0.66      0.72       743
               shelter       0.77      0.55      0.64       570
              clothing       0.68      0.38      0.49       102
                 money       0.55      0.26      0.35       166
        missing_people       0.35      

### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
# get list of model parameters
model.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'features__text_pipeline__vect__tokenizer', 

In [24]:
model_2 = ML_pipeline()

# Model training using GridSearchCV is computational extensive task. 
# Computational time increases as we increase the number of parameters.
# In view of that, only a few parameters are chosen to demonstrate the concept.
# To obtain a highly optimized model, we need to increase the number of parameters.

parameters = {
    'clf__estimator__learning_rate': [0.5, 1.0],
    'clf__estimator__n_estimators': [10, 20]
}


cv = GridSearchCV(model_2, param_grid=parameters, cv=5, n_jobs=-1, verbose=3) 
# verbose=3 to get real time training progress
# n_jobs=-1 -> to train in parallel across the maximum number of cores in our computer, spending it up.
# cv=5 -> 5-fold cross validation


cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estima

[Parallel(n_jobs=-1)]: Done   3 out of  20 | elapsed: 16.5min remaining: 93.6min


[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.20142421159715157, total=13.1min
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20 
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.21795523906408953, total=13.2min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.21515768056968465, total=13.2min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.22126144455747712, total=13.2min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.20594965675057209, total=13.3min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.20981688708036622, total=13.3min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.23906408952187183, total=13.3min


[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed: 16.8min remaining: 16.8min


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.21739130434782608, total=19.5min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.2303585049580473, total=19.5min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.2375381485249237, total=19.7min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.23518942283244343, total=19.7min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.23041709053916581, total=19.6min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.22914547304170904, total=19.8min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.2578184591914569, total= 7.8min


[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed: 24.7min remaining:  4.4min


[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.23677517802644965, total= 7.5min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.24542217700915564, total= 7.6min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.23041709053916581, total= 7.6min


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 24.8min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__learning_rate': [0.5, 1.0], 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [25]:
model_2 = ML_pipeline()

# Model training using GridSearchCV is computational extensive task. 
# Computational time increases as we increase the number of parameters.
# In view of that, only a few parameters are chosen to demonstrate the concept.
# To obtain a highly optimized model, we need to increase the number of parameters.

parameters = {
    'clf__estimator__learning_rate': [0.5, 1.0],
    'clf__estimator__n_estimators': [10, 20]
}


cv = GridSearchCV(model_2, param_grid=parameters, cv=5, n_jobs=-1, verbose=3) 
# verbose=3 to get real time training progress
# n_jobs=-1 -> to train in parallel across the maximum number of cores in our computer.
# cv=5 -> 5-fold cross validation


cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estima

[Parallel(n_jobs=-1)]: Done   3 out of  20 | elapsed: 16.5min remaining: 93.3min


[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20 
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.23906408952187183, total=13.0min
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20 
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.23086702262903636, total=13.0min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.21515768056968465, total=13.1min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10, score=0.22126144455747712, total=13.1min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.20772946859903382, total=13.2min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.20137299771167047, total=13.1min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10, score=0.20142421159715157, total=13.0min


[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed: 16.9min remaining: 16.9min


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.23518942283244343, total=19.4min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.22914547304170904, total=19.3min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.21739130434782608, total=19.2min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.23041709053916581, total=19.1min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.2303585049580473, total=19.5min
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20, score=0.2375381485249237, total=19.4min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.237029501525941, total= 7.0min


[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed: 24.1min remaining:  4.2min


[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.2578184591914569, total= 7.1min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.23041709053916581, total= 6.9min
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=20, score=0.245676500508647, total= 7.1min


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 24.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__learning_rate': [0.5, 1.0], 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
y_pred_test = cv.predict(X_test)

# classification report
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.95      0.88      5012
               request       0.75      0.53      0.62      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.75      0.53      0.62      2717
          medical_help       0.63      0.24      0.35       546
      medical_products       0.62      0.32      0.42       307
     search_and_rescue       0.55      0.22      0.31       179
              security       0.25      0.03      0.05       106
              military       0.67      0.23      0.34       232
                 water       0.67      0.67      0.67       411
                  food       0.80      0.64      0.71       743
               shelter       0.78      0.51      0.61       570
              clothing       0.67      0.41      0.51       102
                 money       0.50      0.18      0.27       166
        missing_people       0.68      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
# trying RandomForestClassifier model 
rf_model = ML_pipeline(clf = RandomForestClassifier())
rf_model.fit(X_train, y_train)
y_pred_rf_test = rf_model.predict(X_test)

print(classification_report(y_test.values, y_pred_rf_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      5012
               request       0.83      0.39      0.53      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.76      0.54      0.63      2717
          medical_help       0.64      0.06      0.12       546
      medical_products       0.58      0.07      0.12       307
     search_and_rescue       0.60      0.02      0.03       179
              security       0.33      0.01      0.02       106
              military       0.58      0.06      0.11       232
                 water       0.83      0.25      0.39       411
                  food       0.86      0.34      0.49       743
               shelter       0.85      0.21      0.34       570
              clothing       0.75      0.06      0.11       102
                 money       0.67      0.02      0.05       166
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
# Trying ML_pipeline_2 which includes custom transformer of 'StartingVerbEstimator'
model_3 = ML_pipeline_2()
model_3.fit(X_train, y_train)
y_pred_3_test = model_3.predict(X_test)

print(classification_report(y_test.values, y_pred_3_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5012
               request       0.75      0.56      0.64      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.77      0.60      0.67      2717
          medical_help       0.63      0.27      0.38       546
      medical_products       0.58      0.35      0.43       307
     search_and_rescue       0.57      0.23      0.33       179
              security       0.16      0.03      0.05       106
              military       0.60      0.33      0.42       232
                 water       0.72      0.62      0.67       411
                  food       0.79      0.66      0.72       743
               shelter       0.77      0.55      0.64       570
              clothing       0.68      0.38      0.49       102
                 money       0.55      0.26      0.35       166
        missing_people       0.35      

### 9. Export your model as a pickle file

In [29]:
# save model in pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(model_3, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.